In [1]:
from langchain_milvus import Milvus
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_core.documents import Document

d:\Demo Projects\rag-chatbot\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# 1) Embeddings
embeddings = HuggingFaceEmbeddings(model_name="BAAI/bge-large-en")

In [5]:
# 2) Milvus Standalone connection (YOUR docker-compose)
vector_store = Milvus(
    embedding_function=embeddings,
    collection_name="financial_docs",
    connection_args={
        # IMPORTANT: HTTP URI to the server, NOT a .db path
        "uri": "http://localhost:19530",
        # if you had auth/db:
        # "user": "username",
        # "password": "password",
        # "db_name": "default",
    },
    index_params={
        "index_type": "HNSW",
        "metric_type": "L2",
        "params": {"M": 8, "efConstruction": 64},
    },
    search_params={
        "metric_type": "L2",
        "params": {"ef": 64},
    },
)


In [6]:
vector_store_saved = Milvus.from_documents(
    [Document(page_content="foo!")],
    embeddings,
    collection_name="langchain_example",
    connection_args={
        "uri": "http://localhost:19530",
    },)

In [7]:
# -------------------------
# 3. Insert test documents
# -------------------------
docs = [
    Document(
        page_content="Apple reported strong quarterly earnings with revenue growth.",
        metadata={"source": "test1.txt"}
    ),
    Document(
        page_content="Microsoft launched new AI-powered features for Azure cloud services.",
        metadata={"source": "test2.txt"}
    ),
    Document(
        page_content="Tesla delivered a record number of vehicles last quarter.",
        metadata={"source": "test3.txt"}
    ),
]

print("Adding documents to vector store...")
vector_store.add_documents(docs)
print("Documents inserted.\n")


# -------------------------
# 4. Run a test query
# -------------------------
query = "Which company had revenue growth?"

print(f"Running search for: {query}")
results = vector_store.similarity_search_with_score(query, k=3)

print("\n---- RESULTS ----\n")

for doc, score in results:
    print("Score:", float(score))
    print("Content:", doc.page_content)
    print("Metadata:", doc.metadata)
    print("--------------------")


Adding documents to vector store...


d:\Demo Projects\rag-chatbot\.venv\lib\site-packages\langchain_milvus\vectorstores\milvus.py:1320: UserWarning: No ids provided and auto_id is False. Setting auto_id to True automatically.
  warnings.warn(


Documents inserted.

Running search for: Which company had revenue growth?

---- RESULTS ----

Score: 0.288912832736969
Content: Apple reported strong quarterly earnings with revenue growth.
Metadata: {'pk': 462188212456009370, 'source': 'test1.txt'}
--------------------
Score: 0.443573534488678
Content: Tesla delivered a record number of vehicles last quarter.
Metadata: {'pk': 462188212456009372, 'source': 'test3.txt'}
--------------------
Score: 0.5393909215927124
Content: Microsoft launched new AI-powered features for Azure cloud services.
Metadata: {'pk': 462188212456009371, 'source': 'test2.txt'}
--------------------


In [11]:
from langchain_milvus.vectorstores import Milvus

embeddings = HuggingFaceEmbeddings(
    model_name="BAAI/bge-large-en"
)

# Connect to Milvus (e.g., Milvus Lite)
# For a full Milvus server, you would provide connection_args like host and port
vector_store = Milvus(
    embedding_function=embeddings,
    connection_args={"uri": URI},
    index_params={"index_type": "FLAT", "metric_type": "L2"},
)

ConnectionConfigException: <ConnectionConfigException: (code=1, message=milvus-lite is required for local database connections. Please install it with: pip install pymilvus[milvus_lite])>

In [6]:
from pymilvus import Collection, MilvusException, connections, db, utility

conn = connections.connect(host="127.0.0.1", port=19530)

# Check if the database exists
db_name = "milvus_demo"
try:
    existing_databases = db.list_database()
    if db_name in existing_databases:
        print(f"Database '{db_name}' already exists.")

        # Use the database context
        db.using_database(db_name)

        # Drop all collections in the database
        collections = utility.list_collections()
        for collection_name in collections:
            collection = Collection(name=collection_name)
            collection.drop()
            print(f"Collection '{collection_name}' has been dropped.")

        db.drop_database(db_name)
        print(f"Database '{db_name}' has been deleted.")
    else:
        print(f"Database '{db_name}' does not exist.")
        database = db.create_database(db_name)
        print(f"Database '{db_name}' created successfully.")
except MilvusException as e:
    print(f"An error occurred: {e}")

MilvusException: <MilvusException: (code=2, message=Fail connecting to server on 127.0.0.1:19530, illegal connection params or server unavailable)>